In [1]:
#
# hw4pr1iris_modeler:  iris clasification via nearest neighbors
#

In [2]:
# libraries!
import numpy as np      # numpy is Python's "array" library
import pandas as pd     # Pandas is Python's "data" library ("dataframe" == spreadsheet)

In [3]:
# let's read in our flower data...
# 
# for read_csv, use header=0 when row 0 is a header row
# 
cleaned_filename = "iris_cleaned.csv"
df_tidy = pd.read_csv(cleaned_filename)   # encoding="utf-8" et al.
print(f"{cleaned_filename} : file read into a pandas dataframe.")
df_tidy

iris_cleaned.csv : file read into a pandas dataframe.


,sepallen,sepalwid,petallen,petalwid,irisname,irisnum
0,4.6,3.6,1.0,0.2,setosa,0
1,4.3,3.0,1.1,0.1,setosa,0
2,5.0,3.2,1.2,0.2,setosa,0
3,5.8,4.0,1.2,0.2,setosa,0
4,4.4,3.0,1.3,0.2,setosa,0
...,...,...,...,...,...,...
136,7.9,3.8,6.4,2.0,virginica,2
137,7.6,3.0,6.6,2.1,virginica,2
138,7.7,3.8,6.7,2.2,virginica,2
139,7.7,2.8,6.7,2.0,virginica,2


In [5]:
#
# Different version vary on how to see all rows (adapt to suit your system!)
# We can see how each line affects our model!
#
# pd.options.display.max_rows = 150   # None for no limit; default: 10
# pd.options.display.min_rows = 150   # None for no limit; default: 10
pd.options.display.max_rows = 10   # None for no limit; default: 10
# pd.options.display.min_rows = 10   # None for no limit; default: 10
df_tidy

,sepallen,sepalwid,petallen,petalwid,irisname,irisnum
0,4.6,3.6,1.0,0.2,setosa,0
1,4.3,3.0,1.1,0.1,setosa,0
2,5.0,3.2,1.2,0.2,setosa,0
3,5.8,4.0,1.2,0.2,setosa,0
4,4.4,3.0,1.3,0.2,setosa,0
...,...,...,...,...,...,...
136,7.9,3.8,6.4,2.0,virginica,2
137,7.6,3.0,6.6,2.1,virginica,2
138,7.7,3.8,6.7,2.2,virginica,2
139,7.7,2.8,6.7,2.0,virginica,2


In [6]:
#
# Let's drop the columns [features] we don't want/need 
#                or that we _shouldn't_ have...!
#

# First, look at the info:
df_tidy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 141 entries, 0 to 140
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   sepallen  141 non-null    float64
 1   sepalwid  141 non-null    float64
 2   petallen  141 non-null    float64
 3   petalwid  141 non-null    float64
 4   irisname  141 non-null    object 
 5   irisnum   141 non-null    int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 7.7+ KB


In [7]:
#
# All of the columns need to be numeric, we'll drop irisname
ROW = 0
COLUMN = 1
df_model1 = df_tidy.drop( 'irisname', axis=COLUMN )
df_model1

,sepallen,sepalwid,petallen,petalwid,irisnum
0,4.6,3.6,1.0,0.2,0
1,4.3,3.0,1.1,0.1,0
2,5.0,3.2,1.2,0.2,0
3,5.8,4.0,1.2,0.2,0
4,4.4,3.0,1.3,0.2,0
...,...,...,...,...,...
136,7.9,3.8,6.4,2.0,2
137,7.6,3.0,6.6,2.1,2
138,7.7,3.8,6.7,2.2,2
139,7.7,2.8,6.7,2.0,2


In [8]:
#
# once we have all the columns we want, let's create an index of their names...

#
# Let's make sure we have all of our helpful variables in one place 
#       To be adapted if we drop/add more columns...
#

#
# let's keep our column names in variables, for reference
#
COLUMNS = df_model1.columns            # "list" of columns
print(f"COLUMNS is {COLUMNS}\n")  
  # It's a "pandas" list, called an Index
  # use it just as a Python list of strings:
print(f"COLUMNS[0] is {COLUMNS[0]}\n")

# let's create a dictionary to look up any column index by name
COL_INDEX = {}
for i, name in enumerate(COLUMNS):
    COL_INDEX[name] = i  # using the name (as key), look up the value (i)
print(f"COL_INDEX is {COL_INDEX}\n\n")


#
# and our "species" names
#

# all of scikit-learn's ML routines need numbers, not strings
#   ... even for categories/classifications (like species!)
#   so, we will convert the flower-species to numbers:

SPECIES = ['setosa','versicolor','virginica']   # int to str
SPECIES_INDEX = {'setosa':0,'versicolor':1,'virginica':2}  # str to int

# Let's try it out...
for name in SPECIES:
    print(f"{name} maps to {SPECIES_INDEX[name]}")

COLUMNS is Index(['sepallen', 'sepalwid', 'petallen', 'petalwid', 'irisnum'], dtype='object')

COLUMNS[0] is sepallen

COL_INDEX is {'sepallen': 0, 'sepalwid': 1, 'petallen': 2, 'petalwid': 3, 'irisnum': 4}


setosa maps to 0
versicolor maps to 1
virginica maps to 2


In [9]:
#
# We _could_ reweight our columns...
# What if petalwid is "worth" 20x more than the others?
# 
df_model1['petalwid'] *= 20
df_model1


,sepallen,sepalwid,petallen,petalwid,irisnum
0,4.6,3.6,1.0,4.0,0
1,4.3,3.0,1.1,2.0,0
2,5.0,3.2,1.2,4.0,0
3,5.8,4.0,1.2,4.0,0
4,4.4,3.0,1.3,4.0,0
...,...,...,...,...,...
136,7.9,3.8,6.4,40.0,2
137,7.6,3.0,6.6,42.0,2
138,7.7,3.8,6.7,44.0,2
139,7.7,2.8,6.7,40.0,2


In [10]:
# Until we have more insight, this seems arbitrary at best -- data-rigging, at worst.
# So, let's set it back...
df_model1['petalwid'] /= 20
df_model1


,sepallen,sepalwid,petallen,petalwid,irisnum
0,4.6,3.6,1.0,0.2,0
1,4.3,3.0,1.1,0.1,0
2,5.0,3.2,1.2,0.2,0
3,5.8,4.0,1.2,0.2,0
4,4.4,3.0,1.3,0.2,0
...,...,...,...,...,...
136,7.9,3.8,6.4,2.0,2
137,7.6,3.0,6.6,2.1,2
138,7.7,3.8,6.7,2.2,2
139,7.7,2.8,6.7,2.0,2


In [11]:
#
A = df_model1.to_numpy()    # yields the underlying numpy array
print(A)


[[4.6 3.6 1.  0.2 0. ]
 [4.3 3.  1.1 0.1 0. ]
 [5.  3.2 1.2 0.2 0. ]
 [5.8 4.  1.2 0.2 0. ]
 [4.4 3.  1.3 0.2 0. ]
 [4.4 3.2 1.3 0.2 0. ]
 [4.5 2.3 1.3 0.3 0. ]
 [4.7 3.2 1.3 0.2 0. ]
 [5.  3.5 1.3 0.3 0. ]
 [5.4 3.9 1.3 0.4 0. ]
 [5.5 3.5 1.3 0.2 0. ]
 [4.4 2.9 1.4 0.2 0. ]
 [4.6 3.4 1.4 0.3 0. ]
 [4.6 3.2 1.4 0.2 0. ]
 [4.8 3.  1.4 0.1 0. ]
 [4.8 3.  1.4 0.3 0. ]
 [4.9 3.  1.4 0.2 0. ]
 [5.  3.6 1.4 0.2 0. ]
 [5.  3.3 1.4 0.2 0. ]
 [5.1 3.5 1.4 0.2 0. ]
 [5.1 3.5 1.4 0.3 0. ]
 [5.2 3.4 1.4 0.2 0. ]
 [5.5 4.2 1.4 0.2 0. ]
 [4.6 3.1 1.5 0.2 0. ]
 [4.9 3.1 1.5 0.1 0. ]
 [4.9 3.1 1.5 0.1 0. ]
 [4.9 3.1 1.5 0.1 0. ]
 [5.  3.4 1.5 0.2 0. ]
 [5.1 3.8 1.5 0.3 0. ]
 [5.1 3.7 1.5 0.4 0. ]
 [5.1 3.4 1.5 0.2 0. ]
 [5.2 3.5 1.5 0.2 0. ]
 [5.3 3.7 1.5 0.2 0. ]
 [5.4 3.7 1.5 0.2 0. ]
 [5.7 4.4 1.5 0.4 0. ]
 [4.7 3.2 1.6 0.2 0. ]
 [4.8 3.4 1.6 0.2 0. ]
 [5.  3.  1.6 0.2 0. ]
 [5.  3.4 1.6 0.4 0. ]
 [5.  3.5 1.6 0.6 0. ]
 [5.1 3.8 1.6 0.2 0. ]
 [5.1 3.3 1.7 0.5 0. ]
 [5.4 3.9 1.7 0.4 0. ]
 [5.4 3.4 1

In [12]:

#
# let's make sure it's all floating-point, so we can multiply and divide
#
A = A.astype('float64')  # so many types:  www.tutorialspoint.com/numpy/numpy_data_types.htm
print(A)


[[4.6 3.6 1.  0.2 0. ]
 [4.3 3.  1.1 0.1 0. ]
 [5.  3.2 1.2 0.2 0. ]
 [5.8 4.  1.2 0.2 0. ]
 [4.4 3.  1.3 0.2 0. ]
 [4.4 3.2 1.3 0.2 0. ]
 [4.5 2.3 1.3 0.3 0. ]
 [4.7 3.2 1.3 0.2 0. ]
 [5.  3.5 1.3 0.3 0. ]
 [5.4 3.9 1.3 0.4 0. ]
 [5.5 3.5 1.3 0.2 0. ]
 [4.4 2.9 1.4 0.2 0. ]
 [4.6 3.4 1.4 0.3 0. ]
 [4.6 3.2 1.4 0.2 0. ]
 [4.8 3.  1.4 0.1 0. ]
 [4.8 3.  1.4 0.3 0. ]
 [4.9 3.  1.4 0.2 0. ]
 [5.  3.6 1.4 0.2 0. ]
 [5.  3.3 1.4 0.2 0. ]
 [5.1 3.5 1.4 0.2 0. ]
 [5.1 3.5 1.4 0.3 0. ]
 [5.2 3.4 1.4 0.2 0. ]
 [5.5 4.2 1.4 0.2 0. ]
 [4.6 3.1 1.5 0.2 0. ]
 [4.9 3.1 1.5 0.1 0. ]
 [4.9 3.1 1.5 0.1 0. ]
 [4.9 3.1 1.5 0.1 0. ]
 [5.  3.4 1.5 0.2 0. ]
 [5.1 3.8 1.5 0.3 0. ]
 [5.1 3.7 1.5 0.4 0. ]
 [5.1 3.4 1.5 0.2 0. ]
 [5.2 3.5 1.5 0.2 0. ]
 [5.3 3.7 1.5 0.2 0. ]
 [5.4 3.7 1.5 0.2 0. ]
 [5.7 4.4 1.5 0.4 0. ]
 [4.7 3.2 1.6 0.2 0. ]
 [4.8 3.4 1.6 0.2 0. ]
 [5.  3.  1.6 0.2 0. ]
 [5.  3.4 1.6 0.4 0. ]
 [5.  3.5 1.6 0.6 0. ]
 [5.1 3.8 1.6 0.2 0. ]
 [5.1 3.3 1.7 0.5 0. ]
 [5.4 3.9 1.7 0.4 0. ]
 [5.4 3.4 1

In [13]:
#
# Also, nice to have NUM_ROWS and NUM_COLS around
#
NUM_ROWS, NUM_COLS = A.shape
print(f"\nThe dataset has {NUM_ROWS} rows and {NUM_COLS} cols")


The dataset has 141 rows and 5 cols


In [14]:
# choose a row index, n:
n = 42
print(f"flower #{n} is {A[n]}")

for i in range(len(COLUMNS)):
    colname = COLUMNS[i]
    value = A[n][i]
    print(f"  Its {colname} is {value}")

species_index = COL_INDEX['irisnum']
species_num = int(round(A[n][species_index]))
species = SPECIES[species_num]
print(f"  Its species is {species} (i.e., {species_num})")


flower #42 is [5.4 3.9 1.7 0.4 0. ]
  Its sepallen is 5.4
  Its sepalwid is 3.9
  Its petallen is 1.7
  Its petalwid is 0.4
  Its irisnum is 0.0
  Its species is setosa (i.e., 0)


In [15]:
print("+++ Start of data definitions +++\n")

#
# we could do this at the data-frame level, too!
#

X_all = A[:,0:4]  # X (features) ... is all rows, columns 0, 1, 2, 3
y_all = A[:,4]    # y (labels) ... is all rows, column 4 only

print(f"y_all (just the labels/species)   are \n {y_all}")
print(f"X_all (just the features) are \n {X_all}")


+++ Start of data definitions +++

y_all (just the labels/species)   are 
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
X_all (just the features) are 
 [[4.6 3.6 1.  0.2]
 [4.3 3.  1.1 0.1]
 [5.  3.2 1.2 0.2]
 [5.8 4.  1.2 0.2]
 [4.4 3.  1.3 0.2]
 [4.4 3.2 1.3 0.2]
 [4.5 2.3 1.3 0.3]
 [4.7 3.2 1.3 0.2]
 [5.  3.5 1.3 0.3]
 [5.4 3.9 1.3 0.4]
 [5.5 3.5 1.3 0.2]
 [4.4 2.9 1.4 0.2]
 [4.6 3.4 1.4 0.3]
 [4.6 3.2 1.4 0.2]
 [4.8 3.  1.4 0.1]
 [4.8 3.  1.4 0.3]
 [4.9 3.  1.4 0.2]
 [5.  3.6 1.4 0.2]
 [5.  3.3 1.4 0.2]
 [5.1 3.5 1.4 0.2]
 [5.1 3.5 1.4 0.3]
 [5.2 3.4 1.4 0.2]
 [5.5 4.2 1.4 0.2]
 [4.6 3.1 1.5 0.2]
 [4.9 3

In [16]:
#
# we scramble the data, to remove (potential) dependence on its ordering: 
# 
indices = np.random.permutation(len(y_all))  # indices is a permutation-list

# we scramble both X and y, necessarily with the same permutation
X_labeled = X_all[indices]              # we apply the _same_ permutation to each!
y_labeled = y_all[indices]              # again...
print(f"The scrambled labels/species are \n {y_labeled}")
print(f"The corresponding data rows are \n {X_labeled}")


The scrambled labels/species are 
 [0. 0. 2. 2. 2. 1. 2. 1. 1. 0. 2. 1. 0. 2. 0. 1. 0. 1. 0. 0. 2. 2. 0. 1.
 2. 1. 2. 1. 2. 0. 2. 0. 2. 2. 0. 2. 1. 0. 2. 0. 0. 1. 1. 0. 2. 1. 0. 1.
 1. 1. 2. 2. 2. 1. 0. 2. 0. 1. 2. 2. 2. 1. 1. 2. 1. 0. 1. 0. 1. 2. 0. 2.
 2. 1. 0. 0. 1. 0. 2. 2. 1. 0. 0. 2. 1. 2. 2. 1. 1. 0. 0. 0. 2. 0. 2. 1.
 1. 0. 1. 0. 2. 0. 0. 2. 1. 0. 1. 0. 1. 0. 0. 0. 2. 1. 0. 2. 0. 2. 0. 2.
 1. 2. 0. 1. 1. 2. 1. 1. 1. 2. 1. 2. 1. 2. 1. 0. 2. 1. 0. 2. 0.]
The corresponding data rows are 
 [[4.6 3.4 1.4 0.3]
 [5.  3.4 1.6 0.4]
 [7.7 2.6 6.9 2.3]
 [7.3 2.9 6.3 1.8]
 [7.7 2.8 6.7 2. ]
 [6.7 3.1 4.7 1.5]
 [7.4 2.8 6.1 1.9]
 [5.5 2.6 4.4 1.2]
 [5.8 2.7 4.1 1. ]
 [5.1 3.8 1.9 0.4]
 [5.7 2.5 5.  2. ]
 [5.  2.3 3.3 1. ]
 [5.4 3.7 1.5 0.2]
 [6.3 3.4 5.6 2.4]
 [5.  3.3 1.4 0.2]
 [5.5 2.3 4.  1.3]
 [4.4 3.  1.3 0.2]
 [6.2 2.2 4.5 1.5]
 [5.5 4.2 1.4 0.2]
 [5.2 3.4 1.4 0.2]
 [5.8 2.8 5.1 2.4]
 [6.4 2.7 5.3 1.9]
 [4.8 3.4 1.6 0.2]
 [6.7 3.1 4.4 1.4]
 [6.9 3.1 5.1 2.3]
 [5.9 3.  4.2 1.5]
 [5.9 3

In [17]:
#
# We next separate into test data and training data ... 
#    + We will train on the training data...
#    + We will _not_ look at the testing data to build the model
#
# Then, afterward, we will test on the testing data -- and see how well we do!
#

#
# a common convention:  train on 80%, test on 20%    Let's define the TEST_PERCENT
#

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2) # random_state=42

print(f"training with {len(y_train)} rows;  testing with {len(y_test)} rows\n" )

print(f"Held-out data... (testing data: {len(y_test)} rows)")
print(f"y_test: {y_test}")
print(f"X_test (first few rows): {X_test[0:5,:]}")  # 5 rows
print()
print(f"Data used for modeling... (training data: {len(y_train)} rows)")
print(f"y_train: {y_train}")
print(f"X_train (first few rows): {X_train[0:5,:]}")  # 5 rows

training with 112 rows;  testing with 29 rows

Held-out data... (testing data: 29 rows)
y_test: [0. 0. 2. 2. 0. 0. 1. 0. 2. 1. 1. 1. 0. 2. 1. 2. 0. 2. 2. 1. 2. 0. 0. 2.
 0. 1. 1. 2. 0.]
X_test (first few rows): [[5.4 3.4 1.7 0.2]
 [5.1 3.4 1.5 0.2]
 [6.5 3.  5.8 2.2]
 [5.7 2.5 5.  2. ]
 [5.5 3.5 1.3 0.2]]

Data used for modeling... (training data: 112 rows)
y_train: [0. 1. 0. 2. 1. 2. 1. 0. 1. 2. 2. 0. 2. 0. 1. 2. 1. 1. 1. 2. 1. 0. 2. 1.
 0. 2. 1. 1. 0. 0. 2. 1. 0. 2. 2. 0. 1. 2. 1. 0. 0. 1. 2. 0. 2. 2. 1. 1.
 2. 0. 2. 1. 2. 1. 0. 1. 2. 1. 0. 0. 0. 0. 0. 2. 2. 0. 0. 1. 2. 1. 0. 0.
 0. 1. 1. 1. 1. 1. 0. 1. 2. 0. 2. 2. 1. 0. 2. 2. 0. 0. 0. 1. 1. 0. 0. 2.
 0. 2. 2. 2. 2. 2. 2. 0. 1. 1. 1. 1. 2. 2. 2. 1.]
X_train (first few rows): [[4.7 3.2 1.3 0.2]
 [5.4 3.  4.5 1.5]
 [4.3 3.  1.1 0.1]
 [6.4 2.8 5.6 2.2]
 [5.9 3.  4.2 1.5]]


<br>

#### Here is where the model-buidling begins in earnest...

In [27]:
#
# +++ This is the "Model-building and Model-training Cell"
#       
# Create a kNN model and train it! 
#
from sklearn.neighbors import KNeighborsClassifier

k = 84   # we don't know what k to use, so we guess!  (this will _not_ be a good value)
knn_model = KNeighborsClassifier(n_neighbors=k)       # here, k is the "k" in kNN

# we train the model (it's one line!)
knn_model.fit(X_train, y_train)                              # yay!  trained!
print("Created and trained a knn classifier with k =", k)  

Created and trained a knn classifier with k = 84


<br>

#### That's it!

In fact, the model is complete!  Let's test...   
<font size="-2">Then, let's build a <i>better</i> model.</font>

In [28]:
#
# +++ This cell is our "Model-testing Cell"
#
# Now, let's see how well our model does on our "held-out data" (the testing data)
#

# We run our test set:

# the function knn_model.predict is the instantiation of our model
# it's what runs the k-nearest-neighbors algorithm:
predicted_labels = knn_model.predict(X_test)      # THIS IS THE KEY LINE:  predict
actual_labels = y_test

# Let's print them so we can compare...
print("Predicted labels:", predicted_labels)
print("Actual  labels  :", actual_labels)

# And, some overall results
num_correct = sum(predicted_labels == actual_labels)
total = len(actual_labels)
print(f"\nResults on test set:  {num_correct} correct out of {total} total.")

Predicted labels: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1.]
Actual  labels  : [0. 0. 2. 2. 0. 0. 1. 0. 2. 1. 1. 1. 0. 2. 1. 2. 0. 2. 2. 1. 2. 0. 0. 2.
 0. 1. 1. 2. 0.]

Results on test set:  8 correct out of 29 total.


In [29]:
#
# Let's print things in a vertical table
#

def compare_labels(predicted_labels, actual_labels):
    """ a more neatly formatted comparison """
    NUM_LABELS = len(predicted_labels)
    num_correct = 0

    print()
    print(f'row {"#":>3s} : {"predicted":>12s} {"actual":<12s}   {"result"}')   
    
    for i in range(NUM_LABELS):
        p = int(round(predicted_labels[i]))         # round protects from fp error 
        a = int(round(actual_labels[i]))
        result = "incorrect"
        if p == a:  # if they match,
            result = ""       # no longer incorrect
            num_correct += 1  # and we count a match!

        print(f"row {i:>3d} : {SPECIES[p]:>12s} {SPECIES[a]:<12s}   {result}")   

    print()
    print("Correct:", num_correct, "out of", NUM_LABELS)
    return num_correct

# let's try it out!
compare_labels(predicted_labels,actual_labels)


row   # :    predicted actual         result
row   0 :   versicolor setosa         incorrect
row   1 :   versicolor setosa         incorrect
row   2 :   versicolor virginica      incorrect
row   3 :   versicolor virginica      incorrect
row   4 :   versicolor setosa         incorrect
row   5 :   versicolor setosa         incorrect
row   6 :   versicolor versicolor     
row   7 :   versicolor setosa         incorrect
row   8 :   versicolor virginica      incorrect
row   9 :   versicolor versicolor     
row  10 :   versicolor versicolor     
row  11 :   versicolor versicolor     
row  12 :   versicolor setosa         incorrect
row  13 :   versicolor virginica      incorrect
row  14 :   versicolor versicolor     
row  15 :   versicolor virginica      incorrect
row  16 :   versicolor setosa         incorrect
row  17 :   versicolor virginica      incorrect
row  18 :   versicolor virginica      incorrect
row  19 :   versicolor versicolor     
row  20 :   versicolor virginica      incorrect


8

In [30]:
#
# Ok!  We have our knn model, we could just use it...

# data-driven predictive model (k-nearest-neighbor), using scikit-learn

# warning: this model has not yet been tuned to its "best k"
#
def predictive_model( Features ):
    """ input: a list of four features 
                [ sepallen, sepalwid, petallen, petalwid ]
        output: the predicted species of iris, from
                  setosa (0), versicolor (1), virginica (2)
    """
    our_features = np.asarray([Features])                 # extra brackets needed
    predicted_species = knn_model.predict(our_features)   # PREDICT!
    
    predicted_species = int(round(predicted_species[0]))  # unpack one element
    name = SPECIES[predicted_species]                     # look up the species
    return name        
    
#
# Try it!
# 
# Features = eval(input("Enter new Features: "))
#
Features = [6.7,3.3,5.7,2.1]            # [5.8,2.7,4.1,1.0] [4.6,3.6,3.0,2.2] [6.7,3.3,5.7,2.1]
result = predictive_model( Features )
print(f"I predict {result} from Features {Features}")

I predict versicolor from Features [6.7, 3.3, 5.7, 2.1]


In [31]:
#
# Except, we didn't really explore whether this was the BEST model we could build...
#
#
# We used k = 84  (a neighborhood size of 84 flowers)
# In a dataset of only 140ish flowers, with three species, this is a _bad_ idea!
#
# Perhaps we should try ALL the neighborhood sizes in their own TRAIN/TEST split
# and see which neighborhood size works the best, for irises, at least...
#
# This is "cross validation" ...
#

In [37]:
#
# Here, we use "cross validation" to find the "best" k...
#

from sklearn.model_selection import cross_val_score

#
# cross-validation splits the training set into two pieces:
#   + model-building and model-validation. We'll use "build" and "validate"
#
best_k = 84  # Not correct!
best_accuracy = 0.0  # also not correct...

# Note that we are cross-validating using only our TEST data!
for k in range(1,85):
    knn_cv_model = KNeighborsClassifier(n_neighbors=k)   # build a knn_model for every k
    cv_scores = cross_val_score( knn_cv_model, X_train, y_train, cv=5 )  # cv=5 means 80/20
    this_cv_accuracy = cv_scores.mean()               # mean() is numpy's built-in average function 
    print(f"k: {k:2d}  cv accuracy: {this_cv_accuracy:7.4f}")

    if this_cv_accuracy > best_accuracy:  # is this one better?
        best_accuracy = this_cv_accuracy  # track the best accuracy
        best_k = k                        # with the best k

    
# use best_k!
print(f"best_k = {best_k}   yields the highest average cv accuracy.")  # print the best one


k:  1  cv accuracy:  0.9285
k:  2  cv accuracy:  0.9194
k:  3  cv accuracy:  0.9466
k:  4  cv accuracy:  0.9198
k:  5  cv accuracy:  0.9289
k:  6  cv accuracy:  0.9375
k:  7  cv accuracy:  0.9379
k:  8  cv accuracy:  0.9375
k:  9  cv accuracy:  0.9292
k: 10  cv accuracy:  0.9375
k: 11  cv accuracy:  0.9553
k: 12  cv accuracy:  0.9553
k: 13  cv accuracy:  0.9466
k: 14  cv accuracy:  0.9379
k: 15  cv accuracy:  0.9466
k: 16  cv accuracy:  0.9379
k: 17  cv accuracy:  0.9553
k: 18  cv accuracy:  0.9462
k: 19  cv accuracy:  0.9375
k: 20  cv accuracy:  0.9375
k: 21  cv accuracy:  0.9375
k: 22  cv accuracy:  0.9462
k: 23  cv accuracy:  0.9553
k: 24  cv accuracy:  0.9466
k: 25  cv accuracy:  0.9375
k: 26  cv accuracy:  0.9462
k: 27  cv accuracy:  0.9285
k: 28  cv accuracy:  0.9198
k: 29  cv accuracy:  0.9198
k: 30  cv accuracy:  0.9198
k: 31  cv accuracy:  0.9285
k: 32  cv accuracy:  0.9194
k: 33  cv accuracy:  0.9194
k: 34  cv accuracy:  0.9194
k: 35  cv accuracy:  0.9194
k: 36  cv accuracy: 

In [38]:
#
# With the best k, we build and train a new model:
#
# Now using best_k instead of the original, randomly-guessed value:   
#
best_k = best_k   # not needed, but nice
from sklearn.neighbors import KNeighborsClassifier
knn_model_tuned = KNeighborsClassifier(n_neighbors=best_k)   # here, we use the best_k!

# we train the model (one line!)
knn_model_tuned.fit(X_train, y_train)                              # yay!  trained!
print(f"Created + trained a knn classifier, now tuned with a (best) k of {best_k}")  

# How does it do?!  The next cell will show...

Created + trained a knn classifier, now tuned with a (best) k of 11


In [34]:
#
# Re-create and re-run the  "Model-testing Cell"     How does it do with best_k?!
#
predicted_labels = knn_model_tuned.predict(X_test)
actual_labels = y_test

# Let's print them so we can compare...
print("Predicted labels:", predicted_labels)
print("Actual labels:", actual_labels)

# And, the overall results
num_correct = sum(predicted_labels == actual_labels)
total = len(actual_labels)
print(f"\nResults on test set:  {num_correct} correct out of {total} total.\n\n")

# Plus, we'll print our nicer table...
compare_labels(predicted_labels,actual_labels)

Predicted labels: [0. 0. 2. 2. 0. 0. 1. 0. 2. 1. 1. 1. 0. 2. 1. 2. 0. 2. 2. 1. 2. 0. 0. 2.
 0. 1. 1. 2. 0.]
Actual labels: [0. 0. 2. 2. 0. 0. 1. 0. 2. 1. 1. 1. 0. 2. 1. 2. 0. 2. 2. 1. 2. 0. 0. 2.
 0. 1. 1. 2. 0.]

Results on test set:  29 correct out of 29 total.



row   # :    predicted actual         result
row   0 :       setosa setosa         
row   1 :       setosa setosa         
row   2 :    virginica virginica      
row   3 :    virginica virginica      
row   4 :       setosa setosa         
row   5 :       setosa setosa         
row   6 :   versicolor versicolor     
row   7 :       setosa setosa         
row   8 :    virginica virginica      
row   9 :   versicolor versicolor     
row  10 :   versicolor versicolor     
row  11 :   versicolor versicolor     
row  12 :       setosa setosa         
row  13 :    virginica virginica      
row  14 :   versicolor versicolor     
row  15 :    virginica virginica      
row  16 :       setosa setosa         
row  17 :    virginica vi

29

In [35]:
#
# Ok!  We tuned our knn modeling to use the "best" value of k...
#
# And, we should now use ALL available data to train our final predictive model:
#
knn_model_final = KNeighborsClassifier(n_neighbors=best_k)     # here, we use the best_k
knn_model_final.fit(X_all, y_all)                              # KEY DIFFERENCE:  we use ALL the data!
print(f"Created + trained a 'final' knn classifier, with a (best) k of {best_k}") 

Created + trained a 'final' knn classifier, with a (best) k of 11


In [36]:
#
# final predictive model (k-nearest-neighbor), with tuned k + ALL data incorporated
#

def predictive_model( Features, Model ):
    """ input: a list of four features 
                [ sepallen, sepalwid, petallen, petalwid ]
        output: the predicted species of iris, from
                  setosa (0), versicolor (1), virginica (2)
    """
    our_features = np.asarray([Features])                 # extra brackets needed
    predicted_species = Model.predict(our_features)       # The model's prediction!
    predicted_species = int(round(predicted_species[0]))  # unpack the extra brackets
    return predicted_species
   
#
# Try it!
# 
# Features = eval(input("Enter new Features: "))
#
Features = [6.7,3.3,5.7,0.1]  # [5.8,2.7,4.1,1.0] [4.6,3.6,3.0,2.2] [6.7,3.3,5.7,2.1]

LoF = [
[4.8, 3.1, 1.6, 0.2 ],
[5.7, 2.9, 4.2, 1.3 ],
[5.8, 2.7, 5.1, 1.9 ],
[5.2, 4.1, 1.5, 0.1 ],
[5.4, 3.4, 1.5, 0.4 ],
[5.1, 2.5, 3.0, 1.1 ],
[6.2, 2.9, 4.3, 1.3 ],
[6.3, 3.3, 6.0, 2.5 ],
[5.7, 2.8, 4.1, 1.3 ],
]

# run on each one:
for Features in LoF:
    predicted_species = predictive_model( Features, knn_model_final )  # pass in the model, too!
    name = SPECIES[predicted_species]
    print(f"I predict {name} from the features {Features}")    

I predict setosa from the features [4.8, 3.1, 1.6, 0.2]
I predict versicolor from the features [5.7, 2.9, 4.2, 1.3]
I predict virginica from the features [5.8, 2.7, 5.1, 1.9]
I predict setosa from the features [5.2, 4.1, 1.5, 0.1]
I predict setosa from the features [5.4, 3.4, 1.5, 0.4]
I predict versicolor from the features [5.1, 2.5, 3.0, 1.1]
I predict versicolor from the features [6.2, 2.9, 4.3, 1.3]
I predict virginica from the features [6.3, 3.3, 6.0, 2.5]
I predict versicolor from the features [5.7, 2.8, 4.1, 1.3]
